In [1]:
import numpy as np
import pandas as pd
import torch
torch.__version__

'1.7.0+cu101'

In [2]:
import extract_vector as ev
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset,DataLoader,random_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [3]:
def clean_train_data(train_data, train_label):
    n = train_data.shape[0]
    m = train_data.shape[1]
    nan_ll = []
    for i in range(n):
        if(np.isnan(np.sum(train_data[i:i+1, :]))):
            nan_ll.append(i)

    train_data = np.delete(train_data, nan_ll, 0)
    train_label = np.delete(train_label, nan_ll, 0)
    return train_data, train_label

def get_train_labels(train_label):
    for i in train_label:
        if(i[0] > 4.5):
            i[0] = 1
        else:
            i[0] = 0

        if(i[1] > 4.5):
            i[1] = 1
        else:
            i[1] = 0
    return train_label

def get_emotion_label(labels):
    emo = []
    for i in labels:
        if(i[0] == 0 and i[1] == 0):
            emo.append(0)
        elif(i[0] == 1 and i[1] == 0):
            emo.append(1)
        elif(i[0] == 0 and i[1] == 1):
            emo.append(2)
        elif(i[0] == 1 and i[1] == 1):
            emo.append(3)
    return emo

In [4]:
class Entropy_Model(nn.Module):

  def __init__(self):
    super().__init__()
    self.linear1 = nn.Linear(14,64)
    self.linear2 = nn.Linear(64,128)
    self.linear3 = nn.Linear(128,256)
    self.linear4 = nn.Linear(256,512)
    self.linear5 = nn.Linear(512,1)
  
  def forward(self,xb):
    out = self.linear1(xb)
    out = F.relu(out)
    out = self.linear2(out)
    out = F.relu(out)
    out = self.linear3(out)
    out = F.relu(out)
    out = self.linear4(out)
    out = F.relu(out)
    out = self.linear5(out)
    out = torch.sigmoid(out)
    return out
  
  def training_step(self,batch):
    features,label = batch
    out = self(features)
    loss = F.binary_cross_entropy(out,label)
    return loss

  def validation_step(self,batch):
    features,label = batch
    out = self(features)
    loss = F.binary_cross_entropy(out,label)
    acc = accuracy(out,label)
    return {"val_loss": loss.detach(),"val_acc": acc}

  def validation_epoch_end(self,outputs):
    batch_loss = [x['val_loss'] for x in outputs]
    epoch_loss = torch.stack(batch_loss).mean()
    batch_acc = [x['val_acc'] for x in outputs]
    epoch_acc = torch.stack(batch_acc).mean()
    return {"val_loss":epoch_loss.item(),"val_acc":epoch_acc.item()}

  def epoch_end(self,num_epoch,results):
    print("num_epoch: {}, train_loss: {:.2f}, val_loss: {:.2f}, val_acc: {:.2f}".format(num_epoch+1,results['train_loss'],results['val_loss'], results['val_acc']))

In [5]:
def accuracy(out,label):
  out = (out>0.5)
  pred = (out == label).sum()
  return pred/out.shape[0]

def evaluate(model,val_loader):
  outputs = [model.validation_step(batch) for batch in val_loader]
  return model.validation_epoch_end(outputs)

def fit(num_epochs,lr,train_loader,val_loader,model,opt_func=torch.optim.Adam):
  optimizer = opt_func(model.parameters(),lr)
  history = []
  for epoch in range(num_epochs):
    train_losses = []
    for batch in train_loader:
      loss = model.training_step(batch)
      train_losses.append(loss)
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

    results = evaluate(model,val_loader)
    train_loss = torch.stack(train_losses).mean().item()
    results['train_loss'] = train_loss
    model.epoch_end(epoch,results)
    history.append(results)
  return history

## Wavelet Entropy


In [6]:
train_data = ev.getWaveletEntropyData()
train_label = ev.getLabelData(type='ValAr')
train_data,train_label = clean_train_data(train_data,train_label)
train_label = get_train_labels(train_label)

In [7]:
train_data = torch.tensor(train_data,dtype=torch.float32)
train_label = torch.tensor(train_label,dtype=torch.float32)

In [8]:
arousal_dataset = TensorDataset(train_data,train_label[:,0].unsqueeze(1))
valence_dataset = TensorDataset(train_data,train_label[:,1].unsqueeze(1))

### Arousal Model


In [9]:
def split_data(dataset):
  test_size = int(len(dataset) * 0.2)
  train_size = len(dataset) - test_size
  train_ds,test_ds = random_split(dataset,[train_size,test_size])
  return train_ds,test_ds

train_arousal,test_arousal = split_data(arousal_dataset)

batch_size = 32
train_loader = DataLoader(train_arousal,batch_size=batch_size,shuffle=True)
test_loader = DataLoader(test_arousal,batch_size = batch_size,shuffle=True)

In [10]:
arousal_model = Entropy_Model()

In [11]:
history = fit(10,0.0001,train_loader,test_loader,arousal_model)

num_epoch: 1, train_loss: 0.68, val_loss: 0.66, val_acc: 0.71
num_epoch: 2, train_loss: 0.66, val_loss: 0.63, val_acc: 0.71
num_epoch: 3, train_loss: 0.64, val_loss: 0.61, val_acc: 0.71
num_epoch: 4, train_loss: 0.64, val_loss: 0.60, val_acc: 0.71
num_epoch: 5, train_loss: 0.64, val_loss: 0.60, val_acc: 0.72
num_epoch: 6, train_loss: 0.64, val_loss: 0.60, val_acc: 0.71
num_epoch: 7, train_loss: 0.64, val_loss: 0.60, val_acc: 0.72
num_epoch: 8, train_loss: 0.64, val_loss: 0.61, val_acc: 0.71
num_epoch: 9, train_loss: 0.64, val_loss: 0.60, val_acc: 0.72
num_epoch: 10, train_loss: 0.64, val_loss: 0.60, val_acc: 0.72


In [12]:
evaluate(arousal_model,test_loader)

{'val_acc': 0.7141304612159729, 'val_loss': 0.6003692150115967}

### Valence Model

In [13]:
def split_data(dataset):
  test_size = int(len(dataset) * 0.2)
  train_size = len(dataset) - test_size
  train_ds,test_ds = random_split(dataset,[train_size,test_size])
  return train_ds,test_ds

train_valence,test_valence = split_data(valence_dataset)

batch_size = 32
train_loader = DataLoader(train_valence,batch_size=batch_size,shuffle=True)
test_loader = DataLoader(test_valence,batch_size = batch_size,shuffle=True)

In [14]:
valence_model = Entropy_Model()

In [15]:
history = fit(10,0.0001,train_loader,test_loader,valence_model)

num_epoch: 1, train_loss: 0.69, val_loss: 0.67, val_acc: 0.63
num_epoch: 2, train_loss: 0.68, val_loss: 0.66, val_acc: 0.63
num_epoch: 3, train_loss: 0.67, val_loss: 0.66, val_acc: 0.63
num_epoch: 4, train_loss: 0.67, val_loss: 0.66, val_acc: 0.63
num_epoch: 5, train_loss: 0.67, val_loss: 0.66, val_acc: 0.63
num_epoch: 6, train_loss: 0.67, val_loss: 0.66, val_acc: 0.63
num_epoch: 7, train_loss: 0.67, val_loss: 0.66, val_acc: 0.64
num_epoch: 8, train_loss: 0.67, val_loss: 0.65, val_acc: 0.64
num_epoch: 9, train_loss: 0.67, val_loss: 0.65, val_acc: 0.64
num_epoch: 10, train_loss: 0.67, val_loss: 0.66, val_acc: 0.63


In [16]:
evaluate(valence_model,test_loader)

{'val_acc': 0.6279891133308411, 'val_loss': 0.6584290266036987}

## Fourier Entropy

In [17]:
train_data = ev.getFourierEntropyData()
train_label = ev.getLabelData(type='ValAr')
train_data,train_label = clean_train_data(train_data,train_label)
train_label = get_train_labels(train_label)

In [18]:
train_data = torch.tensor(train_data,dtype=torch.float32)
train_label = torch.tensor(train_label,dtype=torch.float32)

In [19]:
arousal_dataset = TensorDataset(train_data,train_label[:,0].unsqueeze(1))
valence_dataset = TensorDataset(train_data,train_label[:,1].unsqueeze(1))

### Arousal Model

In [20]:
def split_data(dataset):
  test_size = int(len(dataset) * 0.2)
  train_size = len(dataset) - test_size
  train_ds,test_ds = random_split(dataset,[train_size,test_size])
  return train_ds,test_ds

train_arousal,test_arousal = split_data(arousal_dataset)

batch_size = 32
train_loader = DataLoader(train_arousal,batch_size=batch_size,shuffle=True)
test_loader = DataLoader(test_arousal,batch_size = batch_size,shuffle=True)

In [21]:
arousal_model = Entropy_Model()

In [22]:
history = fit(20,0.0001,train_loader,test_loader,arousal_model)

num_epoch: 1, train_loss: 0.68, val_loss: 0.66, val_acc: 0.69
num_epoch: 2, train_loss: 0.66, val_loss: 0.64, val_acc: 0.69
num_epoch: 3, train_loss: 0.64, val_loss: 0.63, val_acc: 0.69
num_epoch: 4, train_loss: 0.64, val_loss: 0.63, val_acc: 0.68
num_epoch: 5, train_loss: 0.64, val_loss: 0.63, val_acc: 0.68
num_epoch: 6, train_loss: 0.64, val_loss: 0.63, val_acc: 0.68
num_epoch: 7, train_loss: 0.64, val_loss: 0.62, val_acc: 0.69
num_epoch: 8, train_loss: 0.64, val_loss: 0.62, val_acc: 0.69
num_epoch: 9, train_loss: 0.64, val_loss: 0.62, val_acc: 0.69
num_epoch: 10, train_loss: 0.64, val_loss: 0.62, val_acc: 0.69
num_epoch: 11, train_loss: 0.64, val_loss: 0.62, val_acc: 0.69
num_epoch: 12, train_loss: 0.64, val_loss: 0.62, val_acc: 0.68
num_epoch: 13, train_loss: 0.64, val_loss: 0.61, val_acc: 0.70
num_epoch: 14, train_loss: 0.64, val_loss: 0.62, val_acc: 0.69
num_epoch: 15, train_loss: 0.64, val_loss: 0.62, val_acc: 0.69
num_epoch: 16, train_loss: 0.64, val_loss: 0.62, val_acc: 0.69
n

In [23]:
evaluate(arousal_model,test_loader)

{'val_acc': 0.689130425453186, 'val_loss': 0.6211538910865784}

### Valence Model

In [28]:
def split_data(dataset):
  test_size = int(len(dataset) * 0.2)
  train_size = len(dataset) - test_size
  train_ds,test_ds = random_split(dataset,[train_size,test_size])
  return train_ds,test_ds

train_valence,test_valence = split_data(valence_dataset)

batch_size = 32
train_loader = DataLoader(train_valence,batch_size=batch_size,shuffle=True)
test_loader = DataLoader(test_valence,batch_size = batch_size,shuffle=True)

In [29]:
valence_model = Entropy_Model()

In [30]:
history = fit(25,0.0001,train_loader,test_loader,valence_model)

num_epoch: 1, train_loss: 0.69, val_loss: 0.67, val_acc: 0.67
num_epoch: 2, train_loss: 0.68, val_loss: 0.66, val_acc: 0.67
num_epoch: 3, train_loss: 0.68, val_loss: 0.66, val_acc: 0.67
num_epoch: 4, train_loss: 0.68, val_loss: 0.65, val_acc: 0.66
num_epoch: 5, train_loss: 0.68, val_loss: 0.65, val_acc: 0.67
num_epoch: 6, train_loss: 0.68, val_loss: 0.65, val_acc: 0.67
num_epoch: 7, train_loss: 0.68, val_loss: 0.65, val_acc: 0.67
num_epoch: 8, train_loss: 0.68, val_loss: 0.65, val_acc: 0.67
num_epoch: 9, train_loss: 0.68, val_loss: 0.65, val_acc: 0.67
num_epoch: 10, train_loss: 0.68, val_loss: 0.65, val_acc: 0.67
num_epoch: 11, train_loss: 0.68, val_loss: 0.65, val_acc: 0.67
num_epoch: 12, train_loss: 0.68, val_loss: 0.65, val_acc: 0.67
num_epoch: 13, train_loss: 0.68, val_loss: 0.65, val_acc: 0.66
num_epoch: 14, train_loss: 0.68, val_loss: 0.65, val_acc: 0.67
num_epoch: 15, train_loss: 0.68, val_loss: 0.65, val_acc: 0.67
num_epoch: 16, train_loss: 0.68, val_loss: 0.65, val_acc: 0.67
n

In [31]:
evaluate(valence_model,test_loader)

{'val_acc': 0.67527174949646, 'val_loss': 0.6465975046157837}